<h1> CS 595 - Interactive Machine Learning </h1>
<h2> Assignment 8 - Bernoulli Naive Bayes Transparency</h2>
<h2>Rohan Digambar Gawade.</h2>
<h2>A20379951</h2>

<h1><b>DataSet: Bank Direct Marketing DataSet</b></h1>
<h2>Description:</h2> The dataset contains the clients information collected through campaign by portueguese bank institution, so that we would know which clients the bank should target so that the clients will subscribe to the long term deposit scheme  of the bank.
<br>
The dataset consist of 20 attributes and one class label which is the binary valued attribute indicating whether the client has subscribe the scheme or not.
<br>
The dataset has around 41188 records. 
<h2>Features :
Clients Data</h2>
1 - age (numeric)<br>
2 - job : type of job (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")<br>
3 - marital : marital status (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)<br>
4 - education (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")<br>
5 - default: has credit in default? (categorical: "no","yes","unknown")<br>
6 - housing: has housing loan? (categorical: "no","yes","unknown")<br>
7 - loan: has personal loan? (categorical: "no","yes","unknown")
<h2>Features related to the last contact of the current campaign</h2>
8 - contact: contact communication type (categorical: "cellular","telephone")<br> 
9 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")<br>
10 - day_of_week: last contact day of the week (categorical: "mon","tue","wed","thu","fri")<br>
11 - duration: last contact duration, in seconds (numeric).
<h2>Other Features</h2>
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>
14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
15 - poutcome: outcome of the previous marketing campaign (categorical: "failure","nonexistent","success")
<h2>Social and economic context attributes</h2>
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>
17 - cons.price.idx: consumer price index - monthly indicator (numeric)     <br>
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)     <br>
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>
20 - nr.employed: number of employees - quarterly indicator (numeric)
<h2>Output variable (desired target):</h2>
21 - y - has the client subscribed a term deposit? (binary: "yes","no")<br>
<b>Dataset source:</b> http://archive.ics.uci.edu/ml/datasets/Bank+Marketing <br>
<b>Citation:</b> S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from IPython.display import Image 
import pydotplus 
from sklearn import metrics
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
import math

In [202]:
# IMPORTING DATASET
df_bank_addi = pd.read_csv("../bank-additional/bank-additional/bank-additional-full.csv",";")

In [203]:
df_bank_addi.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [204]:
# Removing the missing rows filled with unknown values

df_bank_addi = df_bank_addi[df_bank_addi.contact != 'unknown'] 
df_bank_addi = df_bank_addi[df_bank_addi.job != 'unknown'] 
df_bank_addi = df_bank_addi[df_bank_addi.marital != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.education != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.default != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.housing != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.loan != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.month != 'unknown']
df_bank_addi = df_bank_addi[df_bank_addi.day_of_week != 'unknown']

In [205]:
#Down Sampling majority class to 3859
df_majority = df_bank_addi[df_bank_addi.y == 'no']
df_minority = df_bank_addi[df_bank_addi.y == 'yes']

df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=3859,     # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_bank = pd.concat([df_majority_downsampled, df_minority])
df_bank.head()
# Display new class counts
df_bank.y.value_counts()

yes    3859
no     3859
Name: y, dtype: int64

<h2> Binarizing the features</h2>


In [206]:
transform_list=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
df_bank = pd.get_dummies(df_bank,columns=transform_list,prefix=transform_list)

In [207]:
#Ordering Features
colslist = list(df_bank.columns.values) 
colslist.pop(colslist.index('y')) 
df_bank = df_bank[colslist+['y']]
# df_bank

In [208]:
def binarizeUsingMean(df_bin,attri):
    #print(attri + " " +str(df_bin[attri].mean()))
    mean = df_bin[attri].mean()
    #print(mean)
    strmean = str('%10.2f'%mean)
    new_attr_name = attri+'>='+strmean.lstrip()
    df_bin[attri] = np.where(df_bin[attri] >= mean , 1 , 0)
    df_bin = df_bin.rename(columns={attri : new_attr_name})
    return df_bin
    

In [209]:
#binarize numerical features
#pdays means number of days that passed by after the client was last contacted 
#from a previous campaign (numeric; 999 means client was not previously contacted)
#So I binarized it based on client contacted and client not contacted previously
df_bank['pdays'] = np.where(df_bank['pdays'] == 999, 0 , 1)

binary_col_list = ['age','duration','campaign','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']
for attribute in binary_col_list:
    df_bank = binarizeUsingMean(df_bank,attribute)

In [210]:
df_bank.head(5)

,age>=39.69,duration>=372.01,campaign>=2.32,pdays,previous>=0.33,emp.var.rate>=-0.64,cons.price.idx>=93.44,cons.conf.idx>=-40.29,euribor3m>=2.82,nr.employed>=5129.77,...,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y
34393,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,no
8600,1,0,1,0,0,1,1,0,1,1,...,0,0,0,0,0,1,0,1,0,no
33600,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,no
20641,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,1,0,1,0,no
19612,0,1,0,0,0,1,1,1,1,1,...,0,0,0,1,0,0,0,1,0,no


In [211]:
#Get List of Features
features = list(df_bank.columns[:-1])
#Target
target = df_bank.columns[-1]

X = df_bank[features]
Y = df_bank[target]

In [212]:
#Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.33, random_state = 0)

In [213]:
clf = BernoulliNB()
clf.fit(X_train,y_train)
prob_estimates = clf.predict_proba(X_train)

In [214]:
print(clf.score(X_test,y_test))

0.727915194346


<h2> Getting conditional probability of each feature </h2>

In [215]:
flp =clf.feature_log_prob_
featureprob = np.exp(flp)
# feature with value =1 and class y = no
feature_v_1_y_no = featureprob[0]
# feature with value =1 and class y = yes
feature_v_1_y_yes = featureprob[1]
# taking feature ratio for value = 1 and taking log
feature_ratio_v1 = feature_v_1_y_no/feature_v_1_y_yes
feature_ratio_v1_log = np.log(feature_ratio_v1)

# feature with value = 0 and class y = no
feature_v_0_y_no = 1 - feature_v_1_y_no
# feature with value = 0 and class y = yes
feature_v_0_y_yes = 1 - feature_v_1_y_yes
# taking feature ratio for value = 1 and taking log
feature_ratio_v0 = feature_v_0_y_no/feature_v_0_y_yes
feature_ratio_v0_log = np.log(feature_ratio_v0)
# print(feature_ratio_v0_log)
# print(feature_ratio_v1_log)

In [216]:
# Log probability of the classes.
classprob_no, classprob_yes = np.exp(clf.class_log_prior_)
class_evidence = classprob_no/classprob_yes
class_log_evidence = np.log(class_evidence)
class_log_evidence

-0.0027074083449499082

In [217]:
def get_PosNeg_Evidences(object_ID):
    positive_evidence_set = []
    negative_evidence_set = [] 
    #print("Object "+str(object_ID))
    for feature_i, value_i in enumerate(object_ID):
        #print("Feature "+str(feature_i)+" "+str(value_i))
        if value_i == 0:
            evidence_list.append(feature_ratio_v0_log[feature_i])
            if feature_ratio_v0_log[feature_i] > 0:
                #print(feature_ratio_v0_log[feature_i])
                positive_evidence_set.append(feature_ratio_v0_log[feature_i])
            else :
                #print(feature_ratio_v0_log[feature_i])
                negative_evidence_set.append(feature_ratio_v0_log[feature_i])
        if value_i == 1:
            evidence_list.append(feature_ratio_v1_log[feature_i])
            if feature_ratio_v1_log[feature_i] > 0:
                #print(feature_ratio_v1_log[feature_i])
                positive_evidence_set.append(feature_ratio_v1_log[feature_i])
            else :
                #print(feature_ratio_v1_log[feature_i])
                negative_evidence_set.append(feature_ratio_v1_log[feature_i])

    # Adding the class log evidence to both positive and negative set
    pos_evi = positive_evidence_set
    neg_evi = negative_evidence_set
    pos_evi.append(class_log_evidence)
    neg_evi.append(class_log_evidence)
    total_positive_evidence = sum(pos_evi)
    total_negative_evidence = sum(neg_evi)
    return(total_positive_evidence,total_negative_evidence)

In [218]:
def gettop_PosNeg_Features(object_ID,evidence_list):
    top_features = 3
    #total_pos_evi = np.array(total_pos_evi)
    #print(total_pos_evi)
    top_positive_positions = np.argsort(evidence_list)[-top_features:]
    # print(top_positive_positions)
    #total_neg_evi = np.abs(total_neg_evi)
    top_negative_positions = np.argsort(evidence_list)[:top_features]
    #print(top_negative_positions)
    #print(np.argsort(evidence_list))
    top_pos_features = []
    top_neg_features = []
    feature_nm = np.array(features)
    for i in range(0,3):
        top_pos_features.append({"Feature Name": features[top_positive_positions[i]], "Value ":object_ID[top_positive_positions[i]]})
        top_neg_features.append({"Feature Name": features[top_negative_positions[i]], "Value ":object_ID[top_negative_positions[i]]})
    return (top_pos_features,top_neg_features)

In [219]:
def get_EvidenceInformation(objectIndex,arrayIndex):
    total_Pos_Evidence,total_Neg_Evidence = get_PosNeg_Evidences(objectIndex)
    print("a) the total positive log-evidence = "+str(total_Pos_Evidence))
    print("b) the total negative log-evidence = "+str(total_Neg_Evidence))
    print("c) Probability Distribution = "+str(prob_estimates[arrayIndex,]))
    top_Pos_Feat,top_Neg_Feat = gettop_PosNeg_Features(objectIndex,evidence_list)
    print("d) The top positive features are :\n"+str(top_Pos_Feat))
    print("e )The top negative features are :\n"+str(top_Neg_Feat))

In [220]:
logestimates = clf.predict_log_proba(X_test)
prob_estimates = clf.predict_proba(X_test)
# prob_estimates

<h2>Get following information</h2>
a) the total positive log-evidence<br>  b) the total negative log-evidence<br> c) probability distribution<br> d) top 3 features values that contribute most to the positive evidence<br> e) top 3 feature values that contribute the most to the negative evidence. <br>Print this information on the following object types

<h2>1. The most positive object with respect to the probabilities.</h2>

In [221]:
evidence_list =[]
positive_no_objIndex = np.argmax(prob_estimates[:,0])
# print(X_test.iloc[positive_no_objIndex])
most_pos_obj = X_test.iloc[positive_no_objIndex]
get_EvidenceInformation(most_pos_obj,positive_no_objIndex)

a) the total positive log-evidence = 9.66174494949
b) the total negative log-evidence = -0.471490450643
c) Probability Distribution = [  9.99898257e-01   1.01742682e-04]
d) The top positive features are :
[{'Feature Name': 'nr.employed>=5129.77', 'Value ': 1}, {'Feature Name': 'euribor3m>=2.82', 'Value ': 1}, {'Feature Name': 'emp.var.rate>=-0.64', 'Value ': 1}]
e )The top negative features are :
[{'Feature Name': 'cons.conf.idx>=-40.29', 'Value ': 1}, {'Feature Name': 'day_of_week_mon', 'Value ': 0}, {'Feature Name': 'month_jul', 'Value ': 0}]


<h2>2. The most negative object with respect to the probabilities.</h2>

In [222]:
evidence_list =[]
negative_yes_objIndex = np.argmax(prob_estimates[:,1])
# print(X_train.iloc[negative_yes_objIndex])
most_neg_obj = X_test.iloc[negative_yes_objIndex]
get_EvidenceInformation(most_neg_obj,negative_yes_objIndex)

a) the total positive log-evidence = 0.343217460701
b) the total negative log-evidence = -15.1230459053
c) Probability Distribution = [  3.82276595e-07   9.99999618e-01]
d) The top positive features are :
[{'Feature Name': 'month_apr', 'Value ': 0}, {'Feature Name': 'job_retired', 'Value ': 0}, {'Feature Name': 'month_oct', 'Value ': 0}]
e )The top negative features are :
[{'Feature Name': 'poutcome_success', 'Value ': 1}, {'Feature Name': 'pdays', 'Value ': 1}, {'Feature Name': 'month_mar', 'Value ': 1}]


<h3>3. The object that has the largest positive evidence.</h3>

In [223]:
# find positive evidence for all objects
test_rows = X_test.shape[0]
# test_rows
all_pos_evidence = []
all_neg_evidence = []

for i in range(0,test_rows):
    pos_evidence,neg_evidence = get_PosNeg_Evidences(X_test.iloc[i])
    all_pos_evidence.append(pos_evidence)
    all_neg_evidence.append(neg_evidence)

evidence_list =[]    
most_pos_evi_index = np.argmax(all_pos_evidence)
most_pos_evi_obj = X_test.iloc[most_pos_evi_index]
get_EvidenceInformation(most_pos_evi_obj,most_pos_evi_index)

a) the total positive log-evidence = 9.66174494949
b) the total negative log-evidence = -0.471490450643
c) Probability Distribution = [  9.99898257e-01   1.01742682e-04]
d) The top positive features are :
[{'Feature Name': 'nr.employed>=5129.77', 'Value ': 1}, {'Feature Name': 'euribor3m>=2.82', 'Value ': 1}, {'Feature Name': 'emp.var.rate>=-0.64', 'Value ': 1}]
e )The top negative features are :
[{'Feature Name': 'cons.conf.idx>=-40.29', 'Value ': 1}, {'Feature Name': 'day_of_week_mon', 'Value ': 0}, {'Feature Name': 'month_jul', 'Value ': 0}]


<h3>4. The object that has the largest positive evidence</h3>

In [224]:
evidence_list =[]    
most_neg_evi_index = np.argmin(all_pos_evidence)
most_neg_evi_obj = X_test.iloc[most_neg_evi_index]
get_EvidenceInformation(most_neg_evi_obj,most_neg_evi_index)

a) the total positive log-evidence = 0.343217460701
b) the total negative log-evidence = -15.1230459053
c) Probability Distribution = [  3.82276595e-07   9.99999618e-01]
d) The top positive features are :
[{'Feature Name': 'month_apr', 'Value ': 0}, {'Feature Name': 'job_retired', 'Value ': 0}, {'Feature Name': 'month_oct', 'Value ': 0}]
e )The top negative features are :
[{'Feature Name': 'poutcome_success', 'Value ': 1}, {'Feature Name': 'pdays', 'Value ': 1}, {'Feature Name': 'month_mar', 'Value ': 1}]


<h3> 5. The most uncertain object (the probabilities are closest to 0.5)
</h3>

In [225]:
difference = abs(prob_estimates[:,0])-abs(prob_estimates[:,1])
#print(difference)
uncertain_obj_index = np.abs(difference).argmin()
# print(prob_estimates[uncertain_obj_index]) 
evidence_list =[]    
uncertain_obj = X_test.iloc[uncertain_obj_index]
get_EvidenceInformation(uncertain_obj,uncertain_obj_index)

a) the total positive log-evidence = 4.84677787027
b) the total negative log-evidence = -4.86364987
c) Probability Distribution = [ 0.49645891  0.50354109]
d) The top positive features are :
[{'Feature Name': 'month_may', 'Value ': 1}, {'Feature Name': 'contact_cellular', 'Value ': 0}, {'Feature Name': 'contact_telephone', 'Value ': 1}]
e )The top negative features are :
[{'Feature Name': 'duration>=372.01', 'Value ': 1}, {'Feature Name': 'emp.var.rate>=-0.64', 'Value ': 0}, {'Feature Name': 'euribor3m>=2.82', 'Value ': 0}]
